In [1]:
import bagpy
from bagpy import bagreader
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce
from scipy.spatial.transform import Rotation
import ipympl

%matplotlib widget


In [2]:
sns.set_theme()
# LaTex must be installed for this to work
# sudo apt-get install dvipng texlive-latex-extra texlive-fonts-recommended cm-super

plt.rcParams['text.usetex'] = True
plt.rc('text.latex', preamble=r'\usepackage{bm}')

def drop_columns_in_df(df):
    return df.drop(columns=["header.seq", "header.stamp.secs", "header.stamp.nsecs", "header.frame_id"])

def rename_columns(df, new_colum_names: list):
    column_names = ['Time'] + new_colum_names
    
    df.columns= column_names

    return df

def normalize_df_time(df):
    start_time = df['Time'].iloc[0]

    df['Time'] -= start_time

    return df

def sync_dfs_based_on_time(dataframes: list):
    df_merged = reduce(lambda left,right: pd.merge_asof(left, right, on="Time", allow_exact_matches=True, direction="nearest", tolerance=0.01), dataframes)

    return df_merged

def df_from_pose_to_euler(df):
    quat_df = df[['pose.orientation.x', 'pose.orientation.y', 'pose.orientation.z', 'pose.orientation.w']]
    rot = Rotation.from_quat(quat_df)
    rot_euler = rot.as_euler('xyz', degrees=True)
    euler_df = pd.DataFrame(data=rot_euler, columns=['roll', 'pitch', 'yaw'])

    conc = [df, euler_df]

    df_gt_euler = pd.concat(
        conc,
        axis=1,
        join="outer",
        ignore_index=False,
        keys=None,
        levels=None,
        names=None,
        verify_integrity=False,
        copy=True,
    )

    df_gt_euler = drop_columns_in_df(df_gt_euler)
    df_gt_euler.drop(columns=["pose.position.x", "pose.position.y", "pose.position.z", "pose.orientation.x", "pose.orientation.y","pose.orientation.z","pose.orientation.w"], inplace=True)

    return df_gt_euler

In [3]:
b = bagreader('/home/simenallum/catkin_ws/src/msc_thesis/offline_utils/DJI_0069.bag')

b.topic_table

[INFO]  Successfully created the data folder /home/simenallum/catkin_ws/src/msc_thesis/offline_utils/DJI_0069.


,Topics,Types,Message Count,Frequency
0,/compass_heading,std_msgs/Float32,1484,29.969804
1,/gimbal_heading,std_msgs/Float32,1484,29.969804
2,/gimbal_pitch,std_msgs/Float32,1484,29.969804
3,/gps_data,sensor_msgs/NavSatFix,1484,29.969804
4,/image,sensor_msgs/Image,1484,29.969804
5,/velocity,geometry_msgs/TwistStamped,1484,29.969804


In [17]:
gimbal_heading = b.message_by_topic('/velocity')
df_gimbal_heading = pd.read_csv(gimbal_heading)

# df_gimbal_heading = normalize_df_time(df_gimbal_heading)
# df_gimbal_heading.plot(x="Time", y="data")

df_gimbal_heading

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,twist.linear.x,twist.linear.y,twist.linear.z,twist.angular.x,twist.angular.y,twist.angular.z
0,1.598619e+09,0,1598619002,723599910,NaN,0.199995,0.000000,0.0,0.0,0.0,0.0
1,1.598619e+09,0,1598619002,756967067,NaN,0.199995,0.000000,0.0,0.0,0.0,0.0
2,1.598619e+09,0,1598619002,790333032,NaN,0.199995,0.000000,0.0,0.0,0.0,0.0
3,1.598619e+09,0,1598619002,823699951,NaN,0.499987,0.099997,0.0,0.0,0.0,0.0
4,1.598619e+09,0,1598619002,857067108,NaN,0.499987,0.099997,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1479,1.598619e+09,0,1598619052,72900056,NaN,-3.599903,0.999973,0.0,0.0,0.0,0.0
1480,1.598619e+09,0,1598619052,106266975,NaN,-3.599903,0.999973,0.0,0.0,0.0,0.0
1481,1.598619e+09,0,1598619052,139632940,NaN,-3.899895,1.099970,0.0,0.0,0.0,0.0
1482,1.598619e+09,0,1598619052,173000097,NaN,-3.899895,1.099970,0.0,0.0,0.0,0.0
